In [1]:
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from scipy.optimize import minimize
import matplotlib.pyplot as plt

# 1. Load data (unchanged)
def load_data(filename):
    df = pd.read_csv(filename, index_col=0)
    df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
    years = df.columns.astype(int)
    absolute_data = df.values
    proportions = absolute_data / absolute_data.sum(axis=0)
    return df.index.tolist(), years, proportions

# 2. AS model (unchanged)
def extended_as_model(t, x, s, a, beta):
    n = len(x)
    dxdt = np.zeros(n)
    epsilon = 1e-12
    for i in range(n):
        sum_gain = 0
        sum_loss = 0
        xi = max(x[i], epsilon)
        one_minus_xi = max(1 - xi, epsilon)
        for j in range(n):
            if i != j:
                xj = max(x[j], epsilon)
                one_minus_xj = max(1 - xj, epsilon)
                Pji = s[i] * xi**beta * one_minus_xj**(a - beta)
                Pij = s[j] * xj**beta * one_minus_xi**(a - beta)
                sum_gain += x[j] * Pji
                sum_loss += Pij
        dxdt[i] = sum_gain - x[i] * sum_loss
    return dxdt

# 3. Simulate with tiny yearly shifts
def simulate_language_dynamics(x0, s, a, beta, t_span, t_eval):
    if callable(s):
        s_values = np.array([s(t) for t in t_eval])
    else:
        s_values = np.tile(s, (len(t_eval), 1))

    sol = solve_ivp(extended_as_model, t_span, x0, args=(s_values[0], a, beta),
                    t_eval=[t_span[0]], method='LSODA')

    for i in range(1, len(t_eval)):
        sol_i = solve_ivp(extended_as_model, [t_eval[i-1], t_eval[i]], sol.y[:, -1],
                         args=(s_values[i], a, beta), t_eval=[t_eval[i]], method='LSODA')
        sol.y = np.hstack((sol.y, sol_i.y))
        sol.t = np.hstack((sol.t, sol_i.t))

    return sol.y

# 4. New: Low-volatility status shifts
def gradual_status(t, base_s, volatility=0.01, mean_reversion=0.1):
    """Tiny annual shifts with mean reversion to original status"""
    if not hasattr(gradual_status, 'last_s'):
        gradual_status.last_s = base_s.copy()

    # Tiny random changes (volatility=0.01 is 1% max change)
    delta = volatility * (np.random.random(len(base_s)) - 0.5)  # Centered at 0

    # Mean reversion pulls status back toward original values
    gradual_status.last_s += delta + mean_reversion * (base_s - gradual_status.last_s)

    # Ensure valid probabilities
    gradual_status.last_s = np.clip(gradual_status.last_s, 0.001, 1.0)
    gradual_status.last_s /= np.sum(gradual_status.last_s)

    return gradual_status.last_s

# 5. Loss function
def loss_function_with_si(params, x_data, t_eval):
    a, beta = params[:2]
    s = params[2:]
    s = np.clip(s, 1e-6, 1.0)  # keep s values within [0, 1] to avoid instability
    s = s / np.sum(s)  # normalize so s behaves like probabilities

    x0 = x_data[:, 0]
    x_model = simulate_language_dynamics(x0, s, a, beta, (t_eval[0], t_eval[-1]), t_eval)
    return np.mean((x_model - x_data) ** 2)

# 6. Fit model
def fit_model_per_language(x_data, t_eval):
    n = x_data.shape[0]
    initial_s = np.ones(n) / n
    initial_params = np.concatenate(([1.0, 0.5], initial_s))  # [a, beta, s1, ..., sn]

    bounds = [(0.01, 3.0), (0.01, 3.0)] + [(1e-6, 1.0)] * n  # bounds for a, beta, s_i

    result = minimize(loss_function_with_si, initial_params, args=(x_data, t_eval),
                      bounds=bounds, method='L-BFGS-B')

    a_fit, beta_fit = result.x[:2]
    s_fit = result.x[2:]
    s_fit /= np.sum(s_fit)  # ensure s is normalized
    print(f"Optimization success: {result.success}, Loss: {result.fun}")
    print(f"a = {a_fit:.4f}, beta = {beta_fit:.4f}")
    print("s =", s_fit)
    return a_fit, beta_fit, s_fit

# 7. Updated main function
def main(filename):
    languages, years, proportions = load_data(filename)
    recent_years = years[years >= 2015]
    recent_data = proportions[:, years >= 2015]
    a_fit, beta_fit, s_fit = fit_model_per_language(recent_data, recent_years)

    forecast_start = years[-1]
    future_years = np.arange(forecast_start, forecast_start + 50, 1)

    # Use gradual_status with very low volatility
    s_gradual = lambda t: gradual_status(t, s_fit, volatility=0.005, mean_reversion=0.05)

    # Single simulation (for clean plot)
    x0 = proportions[:, -1]
    x_forecast = simulate_language_dynamics(
        x0, s_gradual, a_fit, beta_fit,
        (forecast_start, forecast_start + 50),
        future_years
    )

    plt.figure(figsize=(12, 6))
    for i, lang in enumerate(languages):
        plt.plot(future_years, x_forecast[i], label=lang, alpha=0.8)
    plt.title('Language Projections with Gradual Status Changes')
    plt.xlabel('Year')
    plt.ylabel('Fraction of Speakers')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, linestyle=':', alpha=0.5)
    plt.tight_layout()
    plt.show()

    # Uncertainty analysis (10 runs)
    n_simulations = 10
    all_simulations = np.zeros((n_simulations, len(languages), len(future_years)))

    for sim in range(n_simulations):
        s_gradual = lambda t: gradual_status(t, s_fit, volatility=0.005)
        all_simulations[sim] = simulate_language_dynamics(
            x0, s_gradual, a_fit, beta_fit,
            (forecast_start, forecast_start + 50),
            future_years
        )

    # Plot median and 25-75th percentiles
    plt.figure(figsize=(12, 6))
    for i, lang in enumerate(languages[:3]):  # First 3 languages
        median = np.median(all_simulations[:, i, :], axis=0)
        p25 = np.percentile(all_simulations[:, i, :], 25, axis=0)
        p75 = np.percentile(all_simulations[:, i, :], 75, axis=0)
        plt.plot(future_years, median, label=lang, lw=2)
        plt.fill_between(future_years, p25, p75, alpha=0.15)

    plt.title('Projections with 25-75th Percentile Bands (Low Volatility)')
    plt.xlabel('Year')
    plt.ylabel('Fraction of Speakers')
    plt.legend()
    plt.show()

main("language_speakers_3.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'language_speakers_3.csv'